In [14]:
import re

def constant_folding(code):
    try:
        op = re.findall('LOC ([0-9]+)\nLOC ([0-9]+)\nMUL', code)
        if op and len(op[0])==2:
            A = int(op[0][0])
            B = int(op[0][1])
            if A!=B:
                code = re.sub('LOC ([0-9]+)\nLOC ([0-9]+)\nMUL', 'LOC ' + str(A * B), code)
        
        op = re.findall('LOC ([0-9]+)\nLOC ([0-9]+)\nADD', code)
        if op and len(op[0])==2:
            A = int(op[0][0])
            B = int(op[0][1])
            if A!=B:
                code = re.sub('LOC ([0-9]+)\nLOC ([0-9]+)\nADD', 'LOC ' + str(A + B), code)
    finally:
        return code

def opr_strength_reduction(code):
    try:
        found = re.findall('LOC 2\nMUL', code)
        if len(found)!=0:
            code = re.sub('LOC 2\nMUL', 'LOC 1\nSHL', code)
    finally:
        return code

def null_seq(code):
    try:
        code = re.sub('ADI 0\n','', code)
        code = re.sub('BEG 0\n','', code)
        code = re.sub('NEG \nNEG','', code)
        code = re.sub('LOC 0\nADD','', code)
        code = re.sub('LOC 0\nSUB','', code)
        code = re.sub('LOC 1\nMUL','', code)
        code = re.sub('LOC 1\nDIV','', code)
    finally:
        return code

def combined_moves(code):
    try:
        op = re.findall('LOV ([0-9]+)\nLOV ([0-9]+)', code)
        if op and len(op[0])==2:
            A = int(op[0][0])
            B = int(op[0][1])
            if (B-A==2):
                code = re.sub('LOV ([0-9]+)\nLOV ([0-9]+)', 'LDV ' + str(A), code)
                
        op = re.findall('LDV ([0-9]+)\nLOV ([0-9]+)', code)
        
        if op and len(op[0])==2:
            A = int(op[0][0])
            B = int(op[0][1])
            if (B-A==4):
                code = re.sub('LDV ([0-9]+)\nLOV ([0-9]+)', 'LAV ' + str(A) + '\nLOI 6', code)
                
    finally:
        return code

def indirect_moves(code):
    try:
        op = re.findall('LAV ([0-9]+)\nLOI 2', code)
        if op:
            A = int(op[0])
            code = re.sub('LAV ([0-9]+)\nLOI 2', 'LOV ' + str(A), code)
        
        op = re.findall('LAV ([0-9]+)\nSTI 2', code)
        if op:
            A = int(op[0])
            code = re.sub('LAV ([0-9]+)\nSTI 2', 'STV ' + str(A), code)
    finally:
        return code

def reordering(code):
    try:
        op = re.findall('ADD\nLOC ([0-9]+)\nADD', code)
        if op:
            A = int(op[0])
            code = re.sub('ADD\nLOC ([0-9]+)\nADD', 'LOC ' + str(A) + '\nADD\nADD' , code)
            
        op = re.findall('ADD\nLOC ([0-9]+)\nSUB', code)
        if op:
            A = int(op[0])
            code = re.sub('ADD\nLOC ([0-9]+)\nSUB', 'LOC ' + str(A) + '\nSUB\nADD' , code)
    finally:
        return code
    
    
code = '''LOC 3
LOC 5
MUL
LOC 6
ADD
ADI 0
LOC 2
MUL
LOV 2
LOV 4
LAV 2
LOI 2
ADD
LOC 2
ADD'''

con_fold_optimized = constant_folding(code)
opr_str_optimized = opr_strength_reduction(con_fold_optimized)
null_seq_optimized = null_seq(opr_str_optimized)
combined_moves_optimized = combined_moves(null_seq_optimized)
indirect_moves_optimized = indirect_moves(combined_moves_optimized)
reordering_optimized = reordering(indirect_moves_optimized)
print reordering_optimized

LOC 21
LOC 1
SHL
LDV 2
LOV 2
LOC 2
ADD
ADD
